In [1]:
#clear all
%reset -f

#import packages
import numpy as np
import sys
import os
import pandas as pd
import mne
import matplotlib
from sklearn.utils import resample
from mne_icalabel import label_components

root = 'F:/Documents/Science/MirRevAdaptEEG'
participants = list(range(0,32))
#specify which erp we are analyzing
erps = 'lrp'

#pop up plots as separate window & interactive
%matplotlib qt
matplotlib.pyplot.close('all')

In [2]:
#setting up path/ directory
#access specific file for specific participant
def load_epochs(pp_num, root_dir, erp_path):

    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'data/eeg/')
    id_directory = os.path.join(data_directory, 'p%03d/' % pp_num)
    pp_directory = os.path.join(id_directory, erp_path)
    filename = os.path.join(pp_directory, 'mra_p%03d_run0-epo.fif' % pp_num)

    epochs = mne.read_epochs(filename)
    #baseline for each trial
    if erp_path == 'frn':
        epochs = epochs.apply_baseline(baseline = (-1.5, -1.2))
    elif erp_path == 'lrp':
        epochs = epochs.apply_baseline(baseline = (-1.3, -1.0))

    return epochs, pp_directory

In [3]:
def direction_index(task, direction, pp):
    if task == 'aln':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\aln_MovementWorkspace_direction.csv')
    elif task == 'rdmrot':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\rdmrot_MovementWorkspace_direction.csv')
    elif task == 'rdmmir':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\rdmmir_MovementWorkspace_direction.csv')
    elif task == 'rot':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\rot_MovementWorkspace_direction.csv')
    elif task == 'mir':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\mir_MovementWorkspace_direction.csv')
    
    ppdata = list(df.iloc[:,pp+1])

    index = [i for i,v in enumerate(ppdata) if direction == v]

    numtrials = len(index)
    
    return index, numtrials

In [4]:
def trial_list_rdm(task, pp, index):
    df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\eeg\p%03d\%s\mra_p%03d_tasktrigindex.csv' % (pp, erps, pp))
    
    #even pp numbers would experience rotation first (need random before this)
    #need to also take into account missing triggers from previous tasks
    if pp % 2 == 0:
        if task == 'rdmrot':
            #get trial after first task (trial 48 to 95, 234 to 281)
            tasksdf = df[0:96]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16150]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)
                    
        elif task == 'rdmmir':
            #get trial after first task (trial 48 to 95)
            tasksdf = df[0:282]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16153]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)
            
    elif pp % 2 == 1:   
        if task == 'rdmrot':
            #get trial after first five tasks (trial 234 to 281)
            tasksdf = df[0:282]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16153]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)
            
        elif task == 'rdmmir':
            #get trial after first task (trial 48 to 95)
            tasksdf = df[0:96]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16150]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

    
    return(triallist)

In [5]:
#get matching trials from eeg data, where we account for missing triggers
def epoch_trials(task, pp, epochs, index):

    df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\eeg\p%03d\%s\mra_p%03d_tasktrigindex.csv' % (pp, erps, pp))
    
    if task == 'aln':
        #get trial numbers where tasktrig is 16149 and trigidx is 1
        tasksdf = df[0:48]
        subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16149]
        triallist = []
        for i in index:
            ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
            nint = int(ndat['trigidx'])
            if nint == 1:
                trial = int(ndat['ctrial'])
                triallist.append(trial)

        epochs = epochs[triallist]
    
    #for the other tasks, the order depends on their pp number
    #even pp numbers would experience rotation first (need random before this)
    #need to also take into account missing triggers from previous tasks
    if pp % 2 == 0:
        if task == 'rot':
            #rot trials are after first two tasks (trial 96 to 185)
            tasksdf = df[0:186]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16151]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]
      
        elif task == 'mir':
            #get trial after first task (trial 282 to 371)
            tasksdf = df[0:372]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16154]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]
            
    elif pp % 2 == 1:
        if task == 'rot':
            #get trial after first five tasks (trial 282 to 371)
            tasksdf = df[0:372]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16151]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]

        elif task == 'mir':
            #get trial after first five tasks (trial 96 to 185)
            tasksdf = df[0:186]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16154]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]
    
    return(epochs)

In [6]:
#save epochs
def output_condition_epochs(pp_num, data, ppdir, condname, task, training):
    # Save the cleaned data :)
    out_fname = os.path.join(ppdir, 'p%03d_EarlyLate_%s_%s_block%s-epo.fif' % (pp_num, condname, task, training))
    data.save(out_fname, overwrite = True)

In [7]:
#save as an evoked object
def output_condition_evoked(pp_num, data, ppdir, condname, task, training):
    evoked = data.average()
    out_fname = os.path.join(ppdir, 'p%03d_EarlyLate_%s_%s_block%s-ave.fif' % (pp_num, condname, task, training))
    evoked.save(out_fname, overwrite = True)

In [8]:
conditions = ['r', 'l']
conditionnames = ['rightmoves', 'leftmoves']
l_cutoff = 30

#ALIGNED
for pp in participants:
    data, ppdir = load_epochs(pp, root_dir = root, erp_path = erps)
    for cond in range(0, len(conditions)):
        diridx, dirtrials = direction_index('aln', conditions[cond], pp)
        aldata = epoch_trials('aln', pp, data, diridx)
        output_condition_epochs(pp, aldata, ppdir, conditionnames[cond], 'aligned', 'all')
        #filter at 30 Hz
        aldata = aldata.filter(l_freq= None, h_freq=l_cutoff)
        #then average across trials
        output_condition_evoked(pp, aldata, ppdir, conditionnames[cond], 'aligned', 'all')
        
#ALIGNED
#Separate according to corresponding participants for rotation and mirror (only half of N do each perturbation type)

aln_rot_participants = [0,1,2,3,8,9,10,11,16,17,18,19,24,25,26,27]
aln_mir_participants = [4,5,6,7,12,13,14,15,20,21,22,23,28,29,30,31]

#aligned for rotation
for pp in list(range(0,len(aln_rot_participants))):
    data, ppdir = load_epochs(aln_rot_participants[pp], root_dir = root, erp_path = erps)
    for cond in range(0, len(conditions)):
        diridx, dirtrials = direction_index('aln', conditions[cond], aln_rot_participants[pp])
        aldata = epoch_trials('aln', aln_rot_participants[pp], data, diridx)
        output_condition_epochs(aln_rot_participants[pp], aldata, ppdir, conditionnames[cond], 'aln_rot', 'all')
        #filter at 30 Hz
        aldata = aldata.filter(l_freq= None, h_freq=l_cutoff)
        #then average across trials
        output_condition_evoked(aln_rot_participants[pp], aldata, ppdir, conditionnames[cond], 'aln_rot', 'all')
        
#aligned for mirror
for pp in list(range(0,len(aln_mir_participants))):
    data, ppdir = load_epochs(aln_mir_participants[pp], root_dir = root, erp_path = erps)
    for cond in range(0, len(conditions)):
        diridx, dirtrials = direction_index('aln', conditions[cond], aln_mir_participants[pp])
        aldata = epoch_trials('aln', aln_mir_participants[pp], data, diridx)
        output_condition_epochs(aln_mir_participants[pp], aldata, ppdir, conditionnames[cond], 'aln_mir', 'all')
        #filter at 30 Hz
        aldata = aldata.filter(l_freq= None, h_freq=l_cutoff)
        #then average across trials
        output_condition_evoked(aln_mir_participants[pp], aldata, ppdir, conditionnames[cond], 'aln_mir', 'all')

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\lrp\mra_p000_run0-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms
        0 CTF compensation matrices available
Not setting metadata
420 matching events found
No baseline correction applied
0 projection items activated
Applying baseline correction (mode: mean)
Overwriting existing file.
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Overwriting existing file.
Overwriting existing file.
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



[Parallel(n_jobs=1)]: Done 768 out of 768 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Overwriting existing file.
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p001\lrp\mra_p001_run0-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms
        0 CTF compensation matrices available


...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Overwriting existing file.
Overwriting existing file.
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



[Parallel(n_jobs=1)]: Done 768 out of 768 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Overwriting existing file.


[Parallel(n_jobs=1)]: Done 768 out of 768 | elapsed:    0.2s finished


In [9]:
#get grand averages - ALIGNED
root_directory = root
data_directory = os.path.join(root_directory, 'data/eeg/')

#read in evoked object
conditionnames = ['rightmoves', 'leftmoves']

#aligned
for condition in range(0, len(conditionnames)):
    flist = []
    evoked_list = []
    for pp in participants:
        id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
        pp_directory = os.path.join(id_directory, erps)
        fname = os.path.join(pp_directory, 'p%03d_EarlyLate_%s_%s_block%s-ave.fif' % (participants[pp], conditionnames[condition], 'aligned', 'all'))
        evoked = mne.read_evokeds(fname)
        flist.append(evoked)
        evoked = flist[pp][0]
        evoked_list.append(evoked)
    if condition == 0:
        aln_right_grand = mne.grand_average(evoked_list)
        aln_right_flist = evoked_list
    elif condition == 1:
        aln_left_grand = mne.grand_average(evoked_list)
        aln_left_flist = evoked_list
        
#aligned for rotation
for condition in range(0, len(conditionnames)):
    flist = []
    evoked_list = []
    for pp in list(range(0,len(aln_rot_participants))):
        id_directory = os.path.join(data_directory, 'p%03d/' % aln_rot_participants[pp])
        pp_directory = os.path.join(id_directory, erps)
        fname = os.path.join(pp_directory, 'p%03d_EarlyLate_%s_%s_block%s-ave.fif' % (aln_rot_participants[pp], conditionnames[condition], 'aln_rot', 'all'))
        evoked = mne.read_evokeds(fname)
        flist.append(evoked)
        evoked = flist[pp][0]
        evoked_list.append(evoked)
    if condition == 0:
        alnrot_right_grand = mne.grand_average(evoked_list)
        alnrot_right_flist = evoked_list
    elif condition == 1:
        alnrot_left_grand = mne.grand_average(evoked_list)
        alnrot_left_flist = evoked_list
        
#aligned for mirror
for condition in range(0, len(conditionnames)):
    flist = []
    evoked_list = []
    for pp in list(range(0,len(aln_rot_participants))):
        id_directory = os.path.join(data_directory, 'p%03d/' % aln_mir_participants[pp])
        pp_directory = os.path.join(id_directory, erps)
        fname = os.path.join(pp_directory, 'p%03d_EarlyLate_%s_%s_block%s-ave.fif' % (aln_mir_participants[pp], conditionnames[condition], 'aln_mir', 'all'))
        evoked = mne.read_evokeds(fname)
        flist.append(evoked)
        evoked = flist[pp][0]
        evoked_list.append(evoked)
    if condition == 0:
        alnmir_right_grand = mne.grand_average(evoked_list)
        alnmir_right_flist = evoked_list
    elif condition == 1:
        alnmir_left_grand = mne.grand_average(evoked_list)
        alnmir_left_flist = evoked_list

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\lrp\p000_EarlyLate_rightmoves_aligned_blockall-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p001\lrp\p001_EarlyLate_rightmoves_aligned_blockall-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p002\lrp\p002_EarlyLate_rightmoves_aligned_blockall-ave.fif ...
    Found the dat

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p019\lrp\p019_EarlyLate_rightmoves_aligned_blockall-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p020\lrp\p020_EarlyLate_rightmoves_aligned_blockall-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p021\lrp\p021_EarlyLate_rightmoves_aligned_blockall-ave.fif ...
    Found the dat

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p006\lrp\p006_EarlyLate_leftmoves_aligned_blockall-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p007\lrp\p007_EarlyLate_leftmoves_aligned_blockall-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p008\lrp\p008_EarlyLate_leftmoves_aligned_blockall-ave.fif ...
    Found the data o

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p025\lrp\p025_EarlyLate_leftmoves_aligned_blockall-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p026\lrp\p026_EarlyLate_leftmoves_aligned_blockall-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p027\lrp\p027_EarlyLate_leftmoves_aligned_blockall-ave.fif ...
    Found the data o

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p024\lrp\p024_EarlyLate_rightmoves_aln_rot_blockall-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p025\lrp\p025_EarlyLate_rightmoves_aln_rot_blockall-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p026\lrp\p026_EarlyLate_rightmoves_aln_rot_blockall-ave.fif ...
    Found the dat

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p027\lrp\p027_EarlyLate_leftmoves_aln_rot_blockall-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Identifying common channels ...
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p004\lrp\p004_EarlyLate_rightmoves_aln_mir_blockall-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p005\lrp\p005_EarlyLate_rightmoves_aln_mir_blockal

Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p006\lrp\p006_EarlyLate_leftmoves_aln_mir_blockall-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p007\lrp\p007_EarlyLate_leftmoves_aln_mir_blockall-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p012\lrp\p012_E

In [10]:
#save as evoked objects
conditions = ['r', 'l']
conditionnames = ['rightmoves', 'leftmoves']
l_cutoff = 30

#ROTATED and MIRROR TRAINING
tasks = ['rot', 'mir']

#split into training blocks
#early (first 45 trials), late (last 45 trials)
b0 = list(range(0,45))
b1 = list(range(45,90))

training = [b0, b1]

       
#rot and mir
for task in range(0, len(tasks)):
    for pp in participants:
        data, ppdir = load_epochs(pp, root_dir = root, erp_path = erps)
        for cond in range(0, len(conditions)):
            for t in range(0, len(training)):
                diridx, dirtrials = direction_index(tasks[task], conditions[cond], pp)
                diridx = [i for i in diridx if i in training[t]]
#                 print(diridx)
                if len(diridx) == 0:
                    break
                pdata = epoch_trials(tasks[task], pp, data, diridx)
#                 print(t, cond)
#                 print(pdata)
                output_condition_epochs(pp, pdata, ppdir, conditionnames[cond], tasks[task], t)
                #filter at 30 Hz
                pdata = pdata.filter(l_freq= None, h_freq=l_cutoff)
                #then average across trials
                output_condition_evoked(pp, pdata, ppdir, conditionnames[cond], tasks[task], t)

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\lrp\mra_p000_run0-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms
        0 CTF compensation matrices available
Not setting metadata
420 matching events found
No baseline correction applied
0 projection items activated
Applying baseline correction (mode: mean)
Overwriting existing file.
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Overwriting existing file.
Overwriting existing file.
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



[Parallel(n_jobs=1)]: Done 1472 out of 1472 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Overwriting existing file.
Overwriting existing file.
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



...

[Parallel(n_jobs=1)]: Done 1280 out of 1280 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Overwriting existing file.
Overwriting existing file.
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



[Parallel(n_jobs=1)]: Done 1408 out of 1408 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Overwriting existing file.


[Parallel(n_jobs=1)]: Done 1600 out of 1600 | elapsed:    0.5s finished


In [11]:
#get grand averages - ROTATED AND MIRROR
root_directory = root
data_directory = os.path.join(root_directory, 'data/eeg/')

#read in evoked object
conditionnames = ['rightmoves', 'leftmoves']
tasks = ['rot', 'mir']
training = [0, 1]
#rot and mir
for task in range(0, len(tasks)):
    for condition in range(0, len(conditionnames)):
        for t in range(0, len(training)):
            flist = []
            evoked_list = []
            for pp in participants:
                id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
                pp_directory = os.path.join(id_directory, erps)
                fname = os.path.join(pp_directory, 'p%03d_EarlyLate_%s_%s_block%s-ave.fif' % (participants[pp], conditionnames[condition], tasks[task], training[t]))
                if os.path.exists(fname) == False:
                    continue
#                 print(fname)
                evoked = mne.read_evokeds(fname)
                flist.append(evoked)
                evoked = evoked[0]
                evoked_list.append(evoked)
#                 print(evoked_list)
            if task == 0 and condition == 0 and t == 0:
                rot_right_b0_grand = mne.grand_average(evoked_list)
                rot_right_b0_flist = evoked_list
            elif task == 0 and condition == 0 and t == 1:
                rot_right_b1_grand = mne.grand_average(evoked_list)
                rot_right_b1_flist = evoked_list
            elif task == 0 and condition == 1 and t == 0:
                rot_left_b0_grand = mne.grand_average(evoked_list)
                rot_left_b0_flist = evoked_list
            elif task == 0 and condition == 1 and t == 1:
                rot_left_b1_grand = mne.grand_average(evoked_list)
                rot_left_b1_flist = evoked_list
            elif task == 1 and condition == 0 and t == 0:
                mir_right_b0_grand = mne.grand_average(evoked_list)
                mir_right_b0_flist = evoked_list
            elif task == 1 and condition == 0 and t == 1:
                mir_right_b1_grand = mne.grand_average(evoked_list)
                mir_right_b1_flist = evoked_list
            elif task == 1 and condition == 1 and t == 0:
                mir_left_b0_grand = mne.grand_average(evoked_list)
                mir_left_b0_flist = evoked_list
            elif task == 1 and condition == 1 and t == 1:
                mir_left_b1_grand = mne.grand_average(evoked_list)
                mir_left_b1_flist = evoked_list

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\lrp\p000_EarlyLate_rightmoves_rot_block0-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 23 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p001\lrp\p001_EarlyLate_rightmoves_rot_block0-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 23 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p002\lrp\p002_EarlyLate_rightmoves_rot_block0-ave.fif ...
    Found the data of interest:
   

    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 22 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p008\lrp\p008_EarlyLate_rightmoves_rot_block1-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 20 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p009\lrp\p009_EarlyLate_rightmoves_rot_block1-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 23 -

        0 CTF compensation matrices available
        nave = 23 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p011\lrp\p011_EarlyLate_leftmoves_rot_block0-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 22 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p016\lrp\p016_EarlyLate_leftmoves_rot_block0-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 22 - aspect type = 100
No projector specified for this dataset. Please consider the met

No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p018\lrp\p018_EarlyLate_leftmoves_rot_block1-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 23 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p019\lrp\p019_EarlyLate_leftmoves_rot_block1-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 22 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] se

Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p029\lrp\p029_EarlyLate_rightmoves_mir_block0-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 22 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p030\lrp\p030_EarlyLate_rightmoves_mir_block0-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 24 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p031\lrp\p031_EarlyLate_r

Identifying common channels ...
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p004\lrp\p004_EarlyLate_leftmoves_mir_block0-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 22 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p005\lrp\p005_EarlyLate_leftmoves_mir_block0-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 26 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p006\lrp\p006_EarlyLate_leftmoves_mir_block0-ave.fif ...
    Fo

    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 22 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p012\lrp\p012_EarlyLate_leftmoves_mir_block1-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 23 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p013\lrp\p013_EarlyLate_leftmoves_mir_block1-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 22 - a

In [12]:
#save as evoked objects
conditions = ['r', 'l']
conditionnames = ['rightmoves', 'leftmoves']
l_cutoff = 30

#RANDOM
tasks = ['rdmrot', 'rdmmir']

#split into training blocks
b0 = list(range(0,24))
b1 = list(range(24,48))

training = [b0, b1]

       
#rdm
#only half will have rdm in rot and other half will have rdm in mir, due to us only considering targets along horizontal axis (i.e., vertical mirror)
for task in range(0, len(tasks)):
    for pp in participants:
        data, ppdir = load_epochs(pp, root_dir = root, erp_path = erps)
        for cond in range(0, len(conditions)):
            for t in range(0, len(training)):
                diridx, dirtrials = direction_index(tasks[task], conditions[cond], pp)
                diridx = [i for i in diridx if i in training[t]]
                if len(diridx) == 0:
                    break
                triallist = trial_list_rdm(tasks[task], pp, diridx)
                pdata = data[triallist]
                print(diridx)
                output_condition_epochs(pp, pdata, ppdir, conditionnames[cond], 'rdm', t)
                #filter at 30 Hz
                pdata = pdata.filter(l_freq= None, h_freq=l_cutoff)
                #then average across trials
                output_condition_evoked(pp, pdata, ppdir, conditionnames[cond], 'rdm', t)

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\lrp\mra_p000_run0-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms
        0 CTF compensation matrices available
Not setting metadata
420 matching events found
No baseline correction applied
0 projection items activated
Applying baseline correction (mode: mean)
[0, 3, 4, 8, 10, 11, 13, 14, 16, 18, 20, 22]
Overwriting existing file.
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Overwriting existing file.
[26, 28, 29, 33, 34, 35, 36, 37, 41, 42, 46, 47]
Overwriting existing file.
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



[Parallel(n_jobs=1)]: Done 768 out of 768 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Overwriting existing file.
[1, 2, 5, 6, 7, 9, 12, 15, 17, 19, 21, 23]
Overwriting existing file.
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



...

[Parallel(n_jobs=1)]: Done 768 out of 768 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Overwriting existing file.
[25, 27, 28, 30, 32, 33, 38, 39, 41, 45, 46, 47]
Overwriting existing file.
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



[Parallel(n_jobs=1)]: Done 768 out of 768 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Overwriting existing file.


[Parallel(n_jobs=1)]: Done 768 out of 768 | elapsed:    0.2s finished


In [13]:
#get grand averages - RANDOM
root_directory = root
data_directory = os.path.join(root_directory, 'data/eeg/')

#read in evoked object
conditionnames = ['rightmoves', 'leftmoves']
tasks = ['rdm']
training = [0, 1]
#rdm
for task in range(0, len(tasks)):
    for condition in range(0, len(conditionnames)):
        for t in range(0, len(training)):
            flist = []
            evoked_list = []
            for pp in participants:
                id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
                pp_directory = os.path.join(id_directory, erps)
                fname = os.path.join(pp_directory, 'p%03d_EarlyLate_%s_%s_block%s-ave.fif' % (participants[pp], conditionnames[condition], tasks[task], training[t]))
                if os.path.exists(fname) == False:
                    continue
#                 print(fname)
                evoked = mne.read_evokeds(fname)
                flist.append(evoked)
                evoked = evoked[0]
                evoked_list.append(evoked)
#                 print(evoked_list)
            if task == 0 and condition == 0 and t == 0:
                rdm_right_b0_grand = mne.grand_average(evoked_list)
                rdm_right_b0_flist = evoked_list
            elif task == 0 and condition == 0 and t == 1:
                rdm_right_b1_grand = mne.grand_average(evoked_list)
                rdm_right_b1_flist = evoked_list
            elif task == 0 and condition == 1 and t == 0:
                rdm_left_b0_grand = mne.grand_average(evoked_list)
                rdm_left_b0_flist = evoked_list
            elif task == 0 and condition == 1 and t == 1:
                rdm_left_b1_grand = mne.grand_average(evoked_list)
                rdm_left_b1_flist = evoked_list

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\lrp\p000_EarlyLate_rightmoves_rdm_block0-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p001\lrp\p001_EarlyLate_rightmoves_rdm_block0-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p002\lrp\p002_EarlyLate_rightmoves_rdm_block0-ave.fif ...
    Found the data of interest:
   

    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p020\lrp\p020_EarlyLate_rightmoves_rdm_block0-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p021\lrp\p021_EarlyLate_rightmoves_rdm_block0-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 -

        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p007\lrp\p007_EarlyLate_rightmoves_rdm_block1-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p008\lrp\p008_EarlyLate_rightmoves_rdm_block1-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the m

No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p026\lrp\p026_EarlyLate_rightmoves_rdm_block1-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p027\lrp\p027_EarlyLate_rightmoves_rdm_block1-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] 

Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p013\lrp\p013_EarlyLate_leftmoves_rdm_block0-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p014\lrp\p014_EarlyLate_leftmoves_rdm_block0-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p015\lrp\p015_EarlyLate_lef

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\lrp\p000_EarlyLate_leftmoves_rdm_block1-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p001\lrp\p001_EarlyLate_leftmoves_rdm_block1-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p002\lrp\p002_EarlyLate_leftmoves_rdm_block1-ave.fif ...
    Found the data of interest:
      

    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p020\lrp\p020_EarlyLate_leftmoves_rdm_block1-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p021\lrp\p021_EarlyLate_leftmoves_rdm_block1-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - a

In [14]:
#runs for LRP ONLY: need separate files for C3 and C4

flists = ['aln_right', 'aln_left', 'alnrot_right', 'alnrot_left', 'alnmir_right', 'alnmir_left',
          'rot_right_b0', 'rot_right_b1', 'rot_left_b0', 'rot_left_b1',
          'mir_right_b0', 'mir_right_b1', 'mir_left_b0', 'mir_left_b1',
          'rdm_right_b0', 'rdm_right_b1', 'rdm_left_b0', 'rdm_left_b1']

channels = ['C3', 'C4']
for channel in channels:
    for f in flists:
        newdf = pd.DataFrame()
        if f == 'aln_right':
            for pp in range(0, len(aln_right_flist)):
                df = aln_right_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_EarlyLate_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'aln_left':
            for pp in range(0, len(aln_left_flist)):
                df = aln_left_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_EarlyLate_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'alnrot_right':
            for pp in range(0, len(alnrot_right_flist)):
                df = alnrot_right_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_EarlyLate_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'alnrot_left':
            for pp in range(0, len(alnrot_left_flist)):
                df = alnrot_left_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_EarlyLate_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'alnmir_right':
            for pp in range(0, len(alnmir_right_flist)):
                df = alnmir_right_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_EarlyLate_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'alnmir_left':
            for pp in range(0, len(alnmir_left_flist)):
                df = alnmir_left_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_EarlyLate_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'rot_right_b0':
            for pp in range(0, len(rot_right_b0_flist)):
                df = rot_right_b0_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_EarlyLate_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'rot_right_b1':
            for pp in range(0, len(rot_right_b1_flist)):
                df = rot_right_b1_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_EarlyLate_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'rot_left_b0':
            for pp in range(0, len(rot_left_b0_flist)):
                df = rot_left_b0_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_EarlyLate_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'rot_left_b1':
            for pp in range(0, len(rot_left_b1_flist)):
                df = rot_left_b1_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_EarlyLate_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'mir_right_b0':
            for pp in range(0, len(mir_right_b0_flist)):
                df = mir_right_b0_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_EarlyLate_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'mir_right_b1':
            for pp in range(0, len(mir_right_b1_flist)):
                df = mir_right_b1_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_EarlyLate_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'mir_left_b0':
            for pp in range(0, len(mir_left_b0_flist)):
                df = mir_left_b0_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_EarlyLate_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'mir_left_b1':
            for pp in range(0, len(mir_left_b1_flist)):
                df = mir_left_b1_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_EarlyLate_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'rdm_right_b0':
            for pp in range(0, len(rdm_right_b0_flist)):
                df = rdm_right_b0_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_EarlyLate_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'rdm_right_b1':
            for pp in range(0, len(rdm_right_b1_flist)):
                df = rdm_right_b1_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_EarlyLate_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'rdm_left_b0':
            for pp in range(0, len(rdm_left_b0_flist)):
                df = rdm_left_b0_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_EarlyLate_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'rdm_left_b1':
            for pp in range(0, len(rdm_left_b1_flist)):
                df = rdm_left_b1_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_EarlyLate_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
             


In [15]:
# # Difference waves between aligned and perturbed conditions for C3
# #channels = ['C3']
# channels = ['C4']
# # channels = ['Oz','O1','O2']
# evokeds = dict(aln_right = right_aln_flist, 
#                aln_left = left_aln_flist,
#                rot_right = right_rot_flist,
#                rot_left = left_rot_flist,
#                rdm_right = right_rdm_flist,
#                rdm_left = left_rdm_flist,
#                mir_right = right_mir_flist,
#                mir_left = left_mir_flist)
# #plot
# mne.viz.plot_compare_evokeds(evokeds, picks=channels, combine = 'mean', legend = 'lower right', ylim=dict(eeg=[-15, 5]), ci=True)